In [37]:
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [22]:
load_dotenv()

True

In [23]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Load document

In [24]:
documents = TextLoader("state_of_the_union.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [25]:
oai_embedding = OpenAIEmbeddings(model="text-embedding-3-small")
cohere_embedding = CohereEmbeddings()
retriever = FAISS.from_documents(texts, cohere_embedding).as_retriever(
    search_kwargs={"k": 20}
)

In [32]:
query = "What did the president say about Ketanji Brown Jackson"

In [33]:
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.
----------------------------------------------------------

## Cohere Reranking

In [34]:
cohere_llm = Cohere(temperature=0)
compressor = CohereRerank(top_n=5, model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

He will never extinguish their love of freedom. He will never weaken the resolve of the free world. 

We meet tonight in an America that has lived through two of the hardest years this nation has ever faced. 

The pandemic has been punishing. 

And so many families are living paycheck to paycheck, struggling to keep up with the rising cost of food, gas, housing, and so much more. 

I understand.
----------------------------------------------------------------------------------------------------
Document 3:

I spoke with their families

In [41]:
chain = RetrievalQA.from_chain_type(
    llm=cohere_llm, retriever=compression_retriever
)

In [42]:
chain({"query": query})
# chain({"question": query})

{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': " In his speech, the president spoke highly of Judge Ketanji Brown Jackson, stating that she is one of the nation's top legal minds and will continue the legacy of excellence of Justice Breyer. He also mentions that she has received broad support from various factions, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans. "}